In [3]:
from bs4 import BeautifulSoup 
import re
from urllib.request import urlopen 
import pandas as pd
from tqdm import tqdm

df = pd.DataFrame(columns=['review_id', 'review_date' ,'review_text', 'agree_score', 'disagree_score',
                             'funny_score', 'professor_id', 'professor_name'])
for i in tqdm(range(20000, 20100)):
    url = 'http://culpa.info/reviews/{}'.format(i)
    r = urlopen(url).read() 
    soup = BeautifulSoup(r, "lxml") 
    # if review doesn't exist, it redirects to home page, maybe use len() to check if we got redirected
    if len(soup.text) > 15500:
        continue

    professor_id = int(soup.find(href=re.compile('(professor)'))['href'][12:])
    professor_name = soup.find(href=re.compile('(professor)')).text
    review_id = int(soup.find(href=re.compile('(reviews)'))['href'][9:])
    review_text = soup.find(class_='review_content').text
    review_date = pd.Timestamp(soup.find(class_='date').text.strip())
    agree_score = int(re.findall('\d+', soup.find(class_='agree')['value'])[0])
    disagree_score = int(re.findall('\d+', soup.find(class_='disagree')['value'])[0])
    funny_score = int(re.findall('\d+', soup.find(class_='funny')['value'])[0])

    to_append = pd.DataFrame(data=[review_id, review_date, review_text, agree_score, 
                              disagree_score, funny_score, professor_id, professor_name],
                            index=['review_id', 'review_date' ,'review_text', 'agree_score', 
                              'disagree_score', 'funny_score', 'professor_id', 'professor_name']).T
    
    df = pd.concat([df, to_append], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.42it/s]


In [9]:
df.professor_id = df.professor_id.apply(int)